<a href="https://colab.research.google.com/github/quinn00/XML-Convention/blob/main/Nomenclature_Conformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Information 

## Overview 

This code is to update the nomenclature of strucutre ID's and associated attributes included in our clinically used XML files. Both structure and protocol templates were updated. 

The ID's and attributes are referenced in a structure set, and there are three other locations just the 

The structure templates contain the structure set, each structure having 5 important assocated attributes that defualt setting when uploaded to a patient. 'ColorandStyle', 'VolumeType', 'Code', 'CodeScheme', and 'CodeSchemeVersion'.

The protocol templates include the same structure set with associated attributes, but also has 3 other locations where the strucutre Id is referenced.
'Item', 'MeasureItem', 'ObjectivesOneStructure' 

Section 1 : Structure Set updates the structure set within either template. A dictionary is created from the 'master_convention' csv file. It is actually a dictionary of dictionaries that when the Id element is found the 5 attributes associated with id in the dictionary is called and changes the template

Section 2 : Protocol Id Reference updates the structure Id located in the 3 different tags. A dictionary is created from the 'protocol_strucutres' csv file, which is based of the 'master_convention' csv. The dictionary refernces the current incorrest Id string and updates it to the desired structure name. 



## Instructions 

I will refer to the file being updated as the target file 

First, our files have to be uploaded to the runtime. The runtime stores files to be able to be refernced within the code. Runtime can be found on the left hand side in the files tab 

Click the folded-corner paper with an up arrow icon or the 'Upload to session storage' button, this will open your library where you can find your target file and open it in the runtime. You will also need to upload the csv dictionaries. Find both 'master_convention' and 'protocol_strucutres' in \\srvnetapp02.vch.ca\bcca\docs\CCSI\XML modifier. 

With both dictionaries and your target files uploaded we are ready to run code

1. First, run the pagkages code as it imports all needed packages

2. Next, pick whether you want to update the structure set elements or the 3 protocol elements. Run 'Strucutre Dictionary' for structure set and 'Protocol Dictionary' for protocol elements. 

3. After running the dictionary cell, delete the file from runtime as the data is now uploaded to the script and we are going to loop through all our files and the csv file will cause an error. 

4. Now section 1 or section 2 are ready to run. Change the name of the output file in the tree.write function. It already uses the name of the original file to name the new one, if you keep the same name it will overwrite the original file in the runtime. Can also be usful to use +'new' or +'updated' to differenciate old files from new

5. Before running the cell, run the Directory Removal cell. Everytime the jupyter notebook is rewritten to the Directory .ipynb_checkpoints. it must be removed in order to avoid the same error we encountered with the dictionary.

6. Run the desired block, Re-fresh the runtime by clicking the 'Refresh' icon and your updated XML files will be waiting for you. 




## Trouble Shoot 

The most reoccuring issue that can happen is due to the juniper notebook framework. If for no reason code won't output but was before or the files are being referenced properly it is because of the runtime. 

The runtime disconnects and sometimes will not prompt you to reconnect. Simply refresh the page or delete the files from the runtime and upload them again 

Other than that issues should not occour as this main file is to be used by everyone and the source should not be changed 

#Common Data

## Packages


In [ ]:
import os 
from xml.etree import ElementTree as ET
import numpy as np
import pandas as pd 
import csv

##Structure Dictionary 

In [ ]:
#this is for the structure set convention. open 'master_convention' to initialize the nomenclauture dictionary 
with open('/content/master_convention.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile))

field_names= [ 'Id', 'Color', 'Volume','Code','Scheme','Version' ]
fc= pd.DataFrame(data, columns=field_names)

dictionary = {} 
for k, id in enumerate(fc['Id']):
    dictionary[id]= {}
    dictionary[id]['Color'] = fc['Color'][k]
    dictionary[id]['Volume'] = fc['Volume'][k]
    dictionary[id]['Code'] = fc['Code'][k]
    dictionary[id]['Scheme']=fc['Scheme'][k]
    dictionary[id]['Version']=fc['Version'][k]

##Protocol Dictionary 

In [ ]:
#this is for protocols. open 'protocol_structures' to initialize the nomenclature dictionary 
with open('/content/protocol_structures.csv', mode='r') as inp:
    reader = csv.reader(inp)
    ref_dict = {rows[0]:rows[1] for rows in reader}

#Directory Removal

In [ ]:
rmdir /content/.ipynb_checkpoints

# Section 1 : Structure Set 

In [ ]:
path = "/content/"
count =-1
d = os.listdir(path) #makes a list of the files in runtime 
del d[0] #deletes defualt stored directories 
del d[-1]
for item in d :
    count+=1
    if count == len(d):
        break
    else:
      tree = ET.parse(d[count])
      root = tree.getroot()
      for j in root.iter("Structure"):
        name= j.get('ID')
        if name.islower() : #capatalizes the first letter of each word if the word is not already capaltalized 
          j.attrib['ID'] = id.title()
#the repetative if statements are specific to our facilty
#fix errors in spelling we have by finding the old spelling and updating it a desired one
        if name == ('Body Only') or name == ('BODY ONLY'):
          j.set('ID','BodyOnly')
        elif name == ('Body'):
          j.set('ID','BODY')
        elif name == ('Opti_PTV_Inter'):
          j.set('ID','OPTI_PTV_Inter')
        elif name== ('OPTI_PTV_Interm'):
          j.set('ID','OPTI_PTV_Inter')
        elif name == ('Opti_PTV_Low'):
          j.set('ID','OPTI_PTV_Low')
        elif name == ('GTV LN'):
          j.set('ID','GTVn')
        elif name == ('Cord PRV4mm'):
          j.set('ID','Cord PRV 4mm')
        elif name== ('GTV Primary'):
          j.set('ID','GTVp')
        if name ==('axillary clips'):
          j.set("ID","Axillary_Clips")
        if name == ('cricoid'):
          j.set('ID','Cricoid')
        if name == ('DEV boost'):
          j.set('ID','DEV Boost')
        if name == ('PTV boost'):
          j.set('ID','PTV Boost')
        if name == ('CTV boost'):
          j.set('ID','CTV Boost')


      for k in root.iter('Structure'):
        id= k.get('ID')
        k.attrib['ID']=id.replace(' ','_') 
      for a in root.iter('Structure'):
        try:
          a.find('Identification').find('StructureCode').attrib['Code']= dictionary[name]['Code']
        except (AttributeError,KeyError):
          continue
        try:
          a.find('ColorAndStyle').text = dictionary[name]['Color']
        except KeyError: 
          continue
          a.find('Identification').find('VolumeType').text=dictionary[name]['Volume']
          a.find('Identification').find('StructureCode').attrib['CodeScheme']= dictionary[name]['Scheme']



      tree.write('new'+ item)

#Section 2 : Protocol Id references 

In [ ]:
path = "/content/"
count =-1
d = os.listdir(path)
print(d)
del d[0]
del d[-1]
print(d)
for item in d :
    count+=1
    if count == len(d):
      break
    else:
      tree = ET.parse(d[count])
      root = tree.getroot()
      for q in root.findall("./Phases/Phase/Prescription/Item"):
        i_id= q.get('ID')
        try:
          q.attrib['ID']=ref_dict[i_id]
        except (KeyError):
          continue

      for r in root.findall("./Phases/Phase/Prescription/MeasureItem"):
        mi_id= r.get('ID')
        try:
          r.attrib['ID']= ref_dict[mi_id]
        except (KeyError):
          continue

      for s in root.findall("./Phases/Phase/ObjectiveTemplate/ObjectivesAllStructures/ObjectivesOneStructure"):
        os_id= s.get('ID')
        try:
          s.attrib['ID']=ref_dict[os_id]
        except (KeyError):
          continue


      tree.write(item)